<a href="https://colab.research.google.com/github/hwarang97/paperswithcode/blob/main/WideResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

!pip install torchinfo
from torchinfo import summary

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cpu


In [ ]:
class Basic_Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels), # pre_activation part
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
        # self.relu = nn.ReLU() # tranfer relu to reisual part

    def forward(self, x):
        residual = self.residual(x)
        if self.in_channels != self.out_channels or self.stride!=1:
            identity = nn.Sequential(
                nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=self.stride, padding=1, bias=False)
                # nn.BatchNorm2d(self.out_channels) # According to pre-activation papaer, because of overlapping BN, remove BN
            )
            shortcut = identity(x)
        else:
            shortcut = x
        # return self.relu(residual+shortcut)
        return residual+shortcut

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.expansion = 4
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels), # pre-activation part(BN, ReLU)
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, self.expansion * out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(self.expansion * out_channels),
        )
        # self.relu = nn.ReLU() # transfer to resiual part

    def forward(self, x):
        residual = self.residual(x)
        if self.in_channels != self.expansion * self.out_channels or self.stride!=1:
            identity = nn.Sequential(
                nn.Conv2d(self.in_channels, self.expansion * self.out_channels, kernel_size=1, stride=self.stride, padding=1, bias=False)
                # nn.BatchNorm2d(self.out_channels)
            )
            shortcut = identity(x)
        else:
            shortcut = x
        # return self.relu(residual+shortcut)
        return residual+shortcut

class ResNet(nn.Module):
    def __init__(self, block, num_layers, num_class=1000):
        super().__init__()
        self.channels = 64

        self.conv_1 = nn.Conv2d(3, 64, kernel_size=7, stride=2)
        self.pool = nn.MaxPool2d(2)
        self.conv_2 = nn.Sequential(
            *self.make_layer(num_layers[0], self.channels, self.channels, block, stride=1)
        )
        self.conv_3 = nn.Sequential(
            *self.make_layer(num_layers[1], block.expansion*self.channels, 2*self.channels, block,stride=2)
        )
        self.conv_4 = nn.Sequential(
            *self.make_layer(num_layers[2], 2*block.expansion*self.channels, 4*self.channels, block,stride=2)
        )
        self.conv_5 = nn.Sequential(
            *self.make_layer(num_layers[3], 4*block.expansion*self.channels, 8*self.channels, block,stride=2)
        )
        self.gap = nn.AvgPool2d(7)
        self.fc = nn.Linear(block.expansion*8*self.channels, num_class)

    def make_layer(self, num, in_channels, out_channels, block, stride):
        layer = []
        if in_channels != block.expansion *out_channels or stride!=1:
            layer.append(nn.Conv2d(in_channels, block.expansion*out_channels, kernel_size=1, stride=stride, padding=0))
            for _ in range(num-1):
                layer.append(block(block.expansion*out_channels, out_channels))
        else:
            for _ in range(num):
                layer.append(block(block.expansion*out_channels, out_channels))
        return layer

    def forward(self, x):
        x = self.conv_1(x)
        x = self.pool(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = self.gap(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

In [ ]:
def resnet18(**kwargs):
    return ResNet(Basic_Block, [2, 2, 2, 2], **kwargs)

def resnet34(**kwargs):
    return ResNet(Basic_Block, [3, 4, 6, 3], **kwargs)

def resnet50(**kwargs):
    return ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)

def resnet101(**kwargs):
    return ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)

def resnet152(**kwargs):
    return ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)

In [ ]:
model = resnet152()
model.train()
summary(model, input_size=(10,3,224,244), device='cpu') # 기본적으로 gpu에 입력값을 생성하기에, cpu로 변경시켜줘야함.

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [10, 1000]                --
├─Conv2d: 1-1                            [10, 64, 109, 119]        9,472
├─MaxPool2d: 1-2                         [10, 64, 54, 59]          --
├─Sequential: 1-3                        [10, 256, 54, 59]         --
│    └─Conv2d: 2-1                       [10, 256, 54, 59]         16,640
│    └─Bottleneck: 2-2                   [10, 256, 54, 59]         --
│    │    └─Sequential: 3-1              [10, 256, 54, 59]         70,912
│    └─Bottleneck: 2-3                   [10, 256, 54, 59]         --
│    │    └─Sequential: 3-2              [10, 256, 54, 59]         70,912
├─Sequential: 1-4                        [10, 512, 27, 30]         --
│    └─Conv2d: 2-4                       [10, 512, 27, 30]         131,584
│    └─Bottleneck: 2-5                   [10, 512, 27, 30]         --
│    │    └─Sequential: 3-3              [10, 512, 27, 30]       